In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import eegraph
import stellargraph as sg
from stellargraph import StellarGraph
import os
import pickle


## Funciones de conectividad y bandas

In [ ]:
def search(values, searchFor):
    for k in values:
        if (searchFor == k):
            return (values[k])

    
def search_key(values, searchFor):
    for key, value in values.items():
        if value[0] == searchFor:
            return key


connectivity_measures = {'squared_coherence': (0, ['theta', 'alpha', 'beta'])}   #CONNECTIVITY MEASURES USED

## Funcion para convertir eegraf a StellarGraph

In [ ]:
def convertir_eegraph(G):
    for i in range(len(G)):
        s = StellarGraph.from_networkx(G[i])
    return s


## funcion modelar Grafo

In [ ]:
def modelate_with_different_connectivity(window_size, label, bands_number, G, conn_empty_values, connectivity_number_total):
    array_grafos = []
    for i in range(connectivity_number_total):
        conn = search_key(connectivity_measures, i)
        bands = search(connectivity_measures, conn)[1]
        
        graphs = G.modelate(window_size = window_size, connectivity = conn, bands = bands)                         #<--------------- THRESHOLD , threshold = 0.2
        array_grafos.append(convertir_eegraph(graphs))
    return  array_grafos

## Abrir directorios e importar datos

In [ ]:
def open_data_directories(path, window_size_class_0, window_size_class_1, bands_number, exclude, connectivity_number_total):
    conn_empty_values = [0] * connectivity_number_total
    array_grafos = [] 
    class_files = os.listdir(path)
    for entry in class_files:
        eeg_files = os.listdir(path + '/' + entry)
        for eeg in eeg_files:
            eeg_path = (path + '/' + entry + '/' + eeg)
            print(eeg_path, entry)
            G = eegraph.Graph() 
            G.load_data(path= eeg_path, exclude = exclude)
            
            if(entry == '1'):
                window_size = window_size_class_1
            elif (entry == '0'):
                window_size = window_size_class_0
            
            array_grafos.append(modelate_with_different_connectivity(window_size, entry ,bands_number, G, conn_empty_values, connectivity_number_total))
        
    return array_grafos

## llamada a las funciones

In [ ]:
path = '../GCNN/Scripts/data/data_vigilia'                                            #<--------------- PATH TO FOLDER CONTAINING EEGs
window_size_class_0 = 1                                 #<--------------- CLASS 0 WINDOW SIZE
window_size_class_1 = 1                               #<--------------- CLASS 1 WINDOW SIZE
connectivity_number_total = 1
bands = 3                                     #<--------------- NUMBER OF FREQUENCY BANDS USED
exclude = [] 
graphs, x1_labels = open_data_directories(path, window_size_class_0, window_size_class_1, bands, exclude, connectivity_number_total)
